## 2012 Fire Polygon TIFF

This notebook takes our final model's predictions for a rectangle around the fire polygon and turns into a TIFF.  This will be run through the  WIFIRE  fire model 

Recall that in order to run through  WIFIRE model, we need LandFire labels, so we have  to convert our categories to LF  types:
* G = 1
* S = 5
* UF = 8
* U = 91
* W = 98
* B = 99
* edges (UG, US, UUF) can be either  Urban (91) or  veg (1, 5, 8)

3 Files are created:
* TIFF of  our  model predictions where "edges" are  called Urban
* TIFF of our model predictions where "edges " are called  Veg
* TIFF of original LANDFIRE labels

In [1]:
import gdal
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def ImportPreserve(FileName,labelName):
    DF=pd.read_table(FileName, header=None)
    DF.columns=['Row']
    DF["Eastern"]=DF["Row"].apply(lambda x: x.split(" ")[0])
    DF["Northern"]=DF["Row"].apply(lambda x: x.split(" ")[1])
    DF[labelName]=DF["Row"].apply(lambda x: x.split(" ")[2])
    del DF["Row"]
    return DF

In [3]:
Ref=pd.read_csv('WIFIRE/Processed_Data/LabelledTiles_Fuel_EqualArea/Reference_Fuel_EqualArea_FirePolygonRectangle2012.csv',dtype=object, index_col=0)
Ref.head()

,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly
0,0,91,33.1262485624999954,-117.182418437500004,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.182266875,33.126097124999994,-117.18257,33.1264
1,1,2,33.1262485624999954,-117.182115312500002,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18196375,33.126097124999994,-117.182266875,33.1264
2,2,2,33.1262485624999954,-117.1818121875,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18166062499999,33.126097124999994,-117.18196375,33.1264
3,3,91,33.1262485624999954,-117.181509062499998,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18135749999999,33.126097124999994,-117.18166062499999,33.1264
4,4,91,33.1262485624999954,-117.181205937499996,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18105437499999,33.126097124999994,-117.18135749999999,33.1264


In [4]:
Pred=pd.read_csv('WIFIRE/Final_Predictions.csv',dtype=object, index_col=0)
Pred.head()

,Unnamed: 0.1,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly,predict_labels
0,0,0,91,33.1262485625,-117.182418438,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.182266875,33.126097125,-117.18257,33.1264,S
1,1,1,2,33.1262485625,-117.182115313,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18196375,33.126097125,-117.182266875,33.1264,UUF
2,2,2,2,33.1262485625,-117.181812188,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.181660625,33.126097125,-117.18196375,33.1264,UUF
3,3,3,91,33.1262485625,-117.181509062,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.1813575,33.126097125,-117.181660625,33.1264,UUF
4,4,4,91,33.1262485625,-117.181205937,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.181054375,33.126097125,-117.1813575,33.1264,UUF


In [12]:
# Assign "edges" as Urban
SwitchLF1={'G':'1','S':'5','UF':'8','UG':'91','US':'91','UUF':'91','U':'91','W':'98','B':'99'}

Pred["PredLabel"]=Pred["predict_labels"].apply(lambda x: SwitchLF1[x])
Pred.head()

,Unnamed: 0.1,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly,predict_labels,PredLabel
0,0,0,91,33.1262485625,-117.182418438,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.182266875,33.126097125,-117.18257,33.1264,S,5
1,1,1,2,33.1262485625,-117.182115313,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18196375,33.126097125,-117.182266875,33.1264,UUF,91
2,2,2,2,33.1262485625,-117.181812188,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.181660625,33.126097125,-117.18196375,33.1264,UUF,91
3,3,3,91,33.1262485625,-117.181509062,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.1813575,33.126097125,-117.181660625,33.1264,UUF,91
4,4,4,91,33.1262485625,-117.181205937,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.181054375,33.126097125,-117.1813575,33.1264,UUF,91


In [13]:
Tiles=pd.merge(Pred[["FuelFileIndex","PredLabel"]],Ref,on='FuelFileIndex',how='left')
print(Tiles[Tiles["Lat"].isnull()].shape[0])

0


In [14]:
Tiles.head()

,FuelFileIndex,PredLabel,Label,Lat,Long,TileName,lrx,lry,ulx,uly
0,0,5,91,33.1262485624999954,-117.182418437500004,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.182266875,33.126097124999994,-117.18257,33.1264
1,1,91,2,33.1262485624999954,-117.182115312500002,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18196375,33.126097124999994,-117.182266875,33.1264
2,2,91,2,33.1262485624999954,-117.1818121875,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18166062499999,33.126097124999994,-117.18196375,33.1264
3,3,91,91,33.1262485624999954,-117.181509062499998,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18135749999999,33.126097124999994,-117.18166062499999,33.1264
4,4,91,91,33.1262485624999954,-117.181205937499996,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18105437499999,33.126097124999994,-117.18135749999999,33.1264


In [17]:
Tiles[["Long", "Lat","PredLabel"]].to_csv('FirePolygonRectangle2012_Predictions_EdgesUrban.xyz',sep=' ', index=False, header=False)


In [18]:
gdal_str="/opt/anaconda3/bin/gdal_translate FirePolygonRectangle2012_Predictions_EdgesUrban.xyz FirePolygonRectangle2012_Predictions_EdgesUrban.tif"
os.system(gdal_str)

0

In [19]:
# Assign "edges" as Veg
SwitchLF1={'G':'1','S':'5','UF':'8','UG':'1','US':'5','UUF':'8','U':'91','W':'98','B':'99'}

Pred["PredLabel"]=Pred["predict_labels"].apply(lambda x: SwitchLF1[x])
Pred.head()

,Unnamed: 0.1,FuelFileIndex,Label,Lat,Long,TileName,lrx,lry,ulx,uly,predict_labels,PredLabel
0,0,0,91,33.1262485625,-117.182418438,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.182266875,33.126097125,-117.18257,33.1264,S,5
1,1,1,2,33.1262485625,-117.182115313,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.18196375,33.126097125,-117.182266875,33.1264,UUF,8
2,2,2,2,33.1262485625,-117.181812188,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.181660625,33.126097125,-117.18196375,33.1264,UUF,8
3,3,3,91,33.1262485625,-117.181509062,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.1813575,33.126097125,-117.181660625,33.1264,UUF,8
4,4,4,91,33.1262485625,-117.181205937,/home/mcmccart/WIFIRE/Processed_Data/LabelledT...,-117.181054375,33.126097125,-117.1813575,33.1264,UUF,8


In [20]:
Tiles=pd.merge(Pred[["FuelFileIndex","PredLabel"]],Ref,on='FuelFileIndex',how='left')
print(Tiles[Tiles["Lat"].isnull()].shape[0])

0


In [21]:
Tiles[["Long", "Lat","PredLabel"]].to_csv('FirePolygonRectangle2012_Predictions_EdgesVeg.xyz',sep=' ', index=False, header=False)



In [22]:
gdal_str="/opt/anaconda3/bin/gdal_translate FirePolygonRectangle2012_Predictions_EdgesVeg.xyz FirePolygonRectangle2012_Predictions_EdgesVeg.tif"
os.system(gdal_str)

0

In [24]:
# Now the original Landfire
Tiles[["Long", "Lat","Label"]].to_csv('FirePolygonRectangle2012_LandFire.xyz',sep=' ', index=False, header=False)

gdal_str="/opt/anaconda3/bin/gdal_translate FirePolygonRectangle2012_LandFire.xyz FirePolygonRectangle2012_LandFire.tif"
os.system(gdal_str)


0